In [1]:
import pandas as pd  
import math
data  = pd.read_csv("sensor_data_200.txt", delimiter=" ", 
                    header=None, names = ("date","time","ir","lidar")).sample(10000).sort_values(by="lidar").reset_index() #lidarの値でソートしておく

lidar_data = pd.DataFrame(data["lidar"])

In [2]:
###負担率の初期化###

K = 2 #クラスタ数
cluster_data_num = int(math.ceil(len(lidar_data)/K)) #クラスタあたりのセンサ値の数
for k in range(K):
    tmp = [1.0 if k == int(i/cluster_data_num) else 0.0 for i,d in lidar_data.iterrows()]
    lidar_data[k] = tmp

In [3]:
lidar_data

,lidar,0,1
0,194,1.0,0.0
1,195,1.0,0.0
2,195,1.0,0.0
3,196,1.0,0.0
4,196,1.0,0.0
5,196,1.0,0.0
6,196,1.0,0.0
7,196,1.0,0.0
8,196,1.0,0.0
9,196,1.0,0.0


In [4]:
def init_parameters(k, min_z, max_z, K):
    step = (max_z - min_z)/K
    return pd.DataFrame({"mu_avg": step*(k + 0.5) + min_z, "zeta":1, "alpha":1, "beta":1, "tau":1}, index=[k])

In [5]:
init_params = pd.concat([init_parameters(k, lidar_data["lidar"].values[0], lidar_data["lidar"].values[-1], K) for k in range(K)])
init_params

,mu_avg,zeta,alpha,beta,tau
0,202.25,1,1,1,1
1,218.75,1,1,1,1


In [6]:
def RST(data, k):
    R = sum([d[k] for _, d in data.iterrows()])
    S = sum([d[k]*d["lidar"] for _, d in data.iterrows()])
    T = sum([d[k]*(d["lidar"]**2) for _, d in data.iterrows()])
    return [R, S, T]

In [7]:
def post_parameters(RST, k):
    R, S, T = RST[k]
    zeta, mu_avg = init_params["zeta"][k],  init_params["mu_avg"][k]
    alpha, beta = init_params["alpha"][k],  init_params["beta"][k]
    tau = init_params["tau"][k]

    tau_post = R + tau
    zeta_post = R + zeta
    mu_avg_post = (S + zeta*mu_avg)/zeta_post
    alpha_post = R/2 + alpha
    beta_post = (T + zeta*(mu_avg**2) - zeta_post*(mu_avg_post**2))/2 + beta
    return pd.DataFrame({"mu_avg": mu_avg_post, "zeta": zeta_post, 
                         "alpha": alpha_post, "beta": beta_post, "tau": tau_post}, index=[k])

In [8]:
def m_step():
    RSTs = [RST(lidar_data, k) for k in range(K)]
    return pd.concat([post_parameters(RSTs, k) for k in range(K)])

In [9]:
params = m_step()

In [10]:
import math
from scipy.special import digamma

def responsibility(d, K, params):
    tau_sum = sum([params["tau"][k] for k in range(K)])
    w = 0.0
    r = {}
    for k in range(K):
        zeta, mu_avg = params["zeta"][k],  params["mu_avg"][k]
        alpha, beta = params["alpha"][k],  params["beta"][k]
        tau = params["tau"][k]
        z = d["lidar"]
        
        log_rho = (digamma(alpha) - math.log(beta))/2 - (1/zeta + ((mu_avg - z)**2)*alpha/beta)/2 + digamma(tau) - digamma(tau_sum)
        r[k] = math.exp(log_rho)
        
        w += r[k]
    for k in range(K):
        r[k] /= w
    
    return r

In [11]:
def e_step(data):
    rs = [responsibility(d, K, params) for _, d in data.iterrows() ]

    for k in range(K):
        data[k] = [rs[i][k] for i,_ in data.iterrows()]
        
    return data

In [12]:
import numpy as np

def one_step(lidar_data):
    params = m_step()
    lidar_data = e_step(lidar_data)
    return params, lidar_data
    
for i in range(1000):
    params, lidar_data = one_step(lidar_data)
    params["stddev_avg"] = np.sqrt(params["beta"]/params["alpha"])
    display(params)

,mu_avg,zeta,alpha,beta,tau,stddev_avg
0,205.737303,5001.0,2501.0,19632.471856,5001.0,2.801758
1,213.716807,5001.0,2501.0,19202.244951,5001.0,2.770889


,mu_avg,zeta,alpha,beta,tau,stddev_avg
0,205.936011,4994.289752,2497.644876,23573.296117,4994.289752,3.072167
1,213.507938,5007.710248,2504.355124,23186.155420,5007.710248,3.042751


,mu_avg,zeta,alpha,beta,tau,stddev_avg
0,205.936011,4994.289752,2497.644876,23573.296117,4994.289752,3.072167
1,213.507938,5007.710248,2504.355124,23186.155420,5007.710248,3.042751


,mu_avg,zeta,alpha,beta,tau,stddev_avg
0,206.045508,4988.323929,2494.661965,25666.113552,4988.323929,3.207556
1,213.389985,5013.676071,2507.338035,25335.412420,5013.676071,3.178759


,mu_avg,zeta,alpha,beta,tau,stddev_avg
0,206.045508,4988.323929,2494.661965,25666.113552,4988.323929,3.207556
1,213.389985,5013.676071,2507.338035,25335.412420,5013.676071,3.178759


...

,mu_avg,zeta,alpha,beta,tau,stddev_avg
0,206.081737,4624.545363,2312.772682,27157.679502,4624.545363,3.426730
1,212.861983,5377.454637,2689.227318,34133.072436,5377.454637,3.562656


,mu_avg,zeta,alpha,beta,tau,stddev_avg
0,206.081737,4624.545363,2312.772682,27157.679502,4624.545363,3.426730
1,212.861983,5377.454637,2689.227318,34133.072436,5377.454637,3.562656


,mu_avg,zeta,alpha,beta,tau,stddev_avg
0,206.080704,4623.075926,2312.037963,27143.399545,4623.075926,3.426373
1,212.861019,5378.924074,2689.962037,34148.752852,5378.924074,3.562988


,mu_avg,zeta,alpha,beta,tau,stddev_avg
0,206.080704,4623.075926,2312.037963,27143.399545,4623.075926,3.426373
1,212.861019,5378.924074,2689.962037,34148.752852,5378.924074,3.562988


KeyboardInterrupt: 

In [ ]:
params

In [ ]:
from scipy.stats import dirichlet
tau = [params["tau"][k] for k in range(K)]

pi = dirichlet(tau).rvs()

In [ ]:
pi